# Import Packages and libraries

In [ ]:
import pandas as pd
import numpy as np
import sklearn
import pycaret
import imblearn as im
from pandas.api.types import CategoricalDtype
from pycaret.classification import *
#from imblearn.over_sampling import ADASYN, BorderlineSMOTE, KMeansSMOTE, RandomOverSampler
#from imblearn.over_sampling import SMOTE,SMOTENC, SVMSMOTE
from imblearn.over_sampling import *
from imblearn.under_sampling import *
from imblearn.combine import SMOTEENN, SMOTETomek
from imblearn.under_sampling import RandomUnderSampler, CondensedNearestNeighbour, EditedNearestNeighbours, RepeatedEditedNearestNeighbours,AllKNN, InstanceHardnessThreshold, NearMiss, NeighbourhoodCleaningRule, OneSidedSelection, TomekLinks, ClusterCentroids
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn')
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
import os
os.environ["GIT_PYTHON_REFRESH"] = "quiet"
import git

# Reading the TAVR Data

In [ ]:
data = pd.read_csv('Data') # reading the csv containing all the data
data.columns = data.columns.str.lower()

data.head() # printing the first five rows of the data

In [ ]:
data = data.dropna(how='any', subset=['race','zipinc_qrtl','elective'])

In [ ]:
data.info()

In [ ]:
#data.shape

In [ ]:
data.drop(data.columns[[0,2,16,17,19,22,24,49,50,56,57,58,59]],axis=1, inplace=True) #All 45 All
data.info()

In [ ]:
#data.shape

In [ ]:
# converting all columns of type object to categorical
data.loc[:, data.dtypes == 'object'] =\
    data.select_dtypes(['object'])\
    .apply(lambda x: x.astype('category'))

In [ ]:
#data.info()

# Research question 2

In [ ]:
train = data[data['year']< 2019]
test = data[data['year']>= 2019]

In [ ]:
#train.shape, test.shape

In [ ]:
#train.to_csv('tarin2019.csv')

In [ ]:
#test.to_csv('test2019.csv')

In [ ]:
#train.info()

In [ ]:
#test.info()

In [ ]:
train.drop(train.columns[[0]],axis=1, inplace=True)
#train.info()

In [ ]:
test.drop(test.columns[[0]],axis=1, inplace=True)
#test.info()

## RandomOverSampler

In [ ]:
ros = RandomOverSampler(sampling_strategy='minority')
ros_setup = setup(data=train,test_data= test, target = 'died', preprocess = True, imputation_type = 'iterative', categorical_imputation = 'constant',
                      handle_unknown_categorical = True, fix_imbalance=True, fix_imbalance_method=ros, fold_strategy = 'stratifiedkfold', fold = 5, n_jobs = -1,
                      session_id=2022, experiment_name='tavr_ros', log_experiment=True ,normalize = True, normalize_method ='minmax',feature_selection= True, remove_multicollinearity= True)

In [ ]:
# adding specificity and balanced accuracy to the computed metrics
add_metric('Spec', 'Spec.', im.metrics.specificity_score)
add_metric('Balanced Accuracy', 'Balanced Accuracy', sklearn.metrics.balanced_accuracy_score)

### Logistic Regression (lr)

In [ ]:
lr = create_model('lr')

In [ ]:
#Tune a Model
tuned_lr = tune_model(lr,optimize = 'Balanced Accuracy')

In [ ]:
#AUC Plot
#plot_model(tuned_lr, plot = 'auc')
#Precision-Recall Curve
#plot_model(tuned_lr, plot = 'pr')
# Feature Importance Plot
#plot_model(tuned_lr, plot='feature')
# Confusion Matrix
#plot_model(tuned_lr, plot = 'confusion_matrix')

#evaluate_model(tuned_lr)

In [ ]:
#Predict on test / hold-out Sample
predict_model(tuned_lr);

In [ ]:
predict_model(lr);

In [ ]:
# Confusion Matrix
#plot_model(tuned_lr, plot = 'confusion_matrix')

In [ ]:
#plot_model(tuned_lr, plot='feature_all')

In [ ]:
#interpret_model(tuned_lr, plot = 'reason', observation = 12)

### Linear Discriminant Analysis(lda)

In [ ]:
lda = create_model('lda')

In [ ]:
#Tune a Model
tuned_lda = tune_model(lda,optimize = 'Balanced Accuracy')

In [ ]:
#AUC Plot
#plot_model(tuned_lda, plot = 'auc')
#Precision-Recall Curve
#plot_model(tuned_lda, plot = 'pr')
# Feature Importance Plot
#plot_model(tuned_lda, plot='feature')
# Confusion Matrix
#plot_model(tuned_lda, plot = 'confusion_matrix')

#evaluate_model(tuned_lda)

In [ ]:
#Predict on test / hold-out Sample
predict_model(tuned_lda);

In [ ]:
predict_model(lda);

In [ ]:
# Confusion Matrix
#plot_model(lda, plot = 'confusion_matrix')

In [ ]:
#plot_model(lda, plot='feature_all')

### Gradient Boosting Classifier (gbc)

In [ ]:
gbc = create_model('gbc',ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=200,
                           n_iter_no_change=None, presort='deprecated', subsample=1.0, tol=0.0001,
                           validation_fraction=0.1,warm_start=False)

In [ ]:
#Tune a Model
tuned_gbc = tune_model(gbc,optimize = 'Balanced Accuracy', n_iter= 100)

In [ ]:
#AUC Plot
#plot_model(tuned_gbc, plot = 'auc')
#Precision-Recall Curve
#plot_model(tuned_gbc, plot = 'pr')
# Feature Importance Plot
#plot_model(tuned_gbc, plot='feature')
# Confusion Matrix
#plot_model(tuned_gbc, plot = 'confusion_matrix')

#evaluate_model(tuned_gbc)

In [ ]:
#Predict on test / hold-out Sample
predict_model(tuned_gbc);

In [ ]:
predict_model(gbc);

In [ ]:
# Confusion Matrix
#plot_model(tuned_gbc, plot = 'confusion_matrix')

In [ ]:
#plot_model(tuned_gbc, plot='feature_all')

### CatBoost Classifier (catboost)

In [ ]:
#catboost = create_model('catboost',class_weights={0:6, 1:9},max_depth=3)
catboost = create_model('catboost')

In [ ]:
#Tune a Model
tuned_catboost = tune_model(catboost,optimize = 'Balanced Accuracy',n_iter= 100)

In [ ]:
#AUC Plot
#plot_model(tuned_catboost, plot = 'auc')
#Precision-Recall Curve
#plot_model(tuned_catboost, plot = 'pr')
# Feature Importance Plot
#plot_model(tuned_catboost, plot='feature')
# Confusion Matrix
#plot_model(tuned_catboost, plot = 'confusion_matrix')

#evaluate_model(tuned_catboost)

In [ ]:
#Predict on test / hold-out Sample
predict_model(tuned_catboost);

In [ ]:
predict_model(catboost);

In [ ]:
# Confusion Matrix
#plot_model(catboost, plot = 'confusion_matrix')

In [ ]:
#plot_model(catboost, plot='feature_all')

### Light Gradient Boosting Machine (lightgbm)

In [ ]:
lightgbm = create_model('lightgbm',class_weight={0:6, 1:9},max_depth=3,num_leaves=80)

In [ ]:
#Tune a Model
tuned_lightgbm = tune_model(lightgbm,optimize = 'Balanced Accuracy',n_iter= 100)

In [ ]:
#AUC Plot
#plot_model(tuned_lightgbm, plot = 'auc')
#Precision-Recall Curve
#plot_model(tuned_lightgbm, plot = 'pr')
# Feature Importance Plot
#plot_model(tuned_lightgbm, plot='feature')
# Confusion Matrix
#plot_model(tuned_lightgbm, plot = 'confusion_matrix')

#evaluate_model(tuned_lightgbm)

In [ ]:
#Predict on test / hold-out Sample
predict_model(tuned_lightgbm);

In [ ]:
predict_model(lightgbm);

In [ ]:
# Confusion Matrix
#plot_model(tuned_lightgbm, plot = 'confusion_matrix')

In [ ]:
#plot_model(tuned_lightgbm, plot='feature_all')